In [1]:
import pandas as pd
import numpy as np
import regex as re

In [2]:
all_drinks = pd.read_pickle('./ing_meas_processed_alcohol.pkl')
compared_alcohol = pd.read_pickle('./preprocessed_alcohol.pkl')

#### 문제점

L 단위의 경우 l 한 글자여서, 필터에서 문자들의 l을 가져와 버리는 문제가 있다.
해당 글자를 기존 데이터와 비교하여서 처리할 필요가 있다.

이는 gr도 마찬가지이다. 단위가 2개 이상 포함되는 경우 뒤의 단위를 삭제하는 알고리즘이 필요하다.

(ex) 'oz', 'l' 이렇게 검색해 보면 'l'이 chilled와 같이 단어에 l이 들어간 경우로 매칭된 것을 알 수 있다.

In [3]:
# verify measure preprocess   # remeber index 74, 85, 103, 163 ...
for i in range(len(all_drinks)):
    print("index: ", i)
    print(all_drinks['Measures'][i])
    print(compared_alcohol['Measures'][i])
    print()

index:  0
[[['1', 'oz'], 1], [['1', 'oz'], 2]]
[['1 oz white ', 1], ['1 oz ', 2]]

index:  1
[[['1/2', 'oz'], 1], [['1/4', 'oz'], 2], [['1/4', 'oz'], 3], [['1/4', 'oz'], 4], [['1/4', 'oz'], 5], [['1/4', 'oz'], 6], [['1/2', 'oz'], 7], [['1/4', 'oz'], 8]]
[['1/2 oz ', 1], ['1/4 oz ', 2], ['1/4 oz ', 3], ['1/4 oz ', 4], ['1/4 oz ', 5], ['1/4 oz ', 6], ['1/2 oz ', 7], ['1/4 oz ', 8]]

index:  2
[[['16', 'oz'], 1], [['1.5', 'oz'], 2]]
[['16 oz ', 1], ['1.5 oz ', 2]]

index:  3
[[['1/2', 'oz'], 1], [['1/2', 'oz'], 2], [['1/2', 'oz'], 3], [['1', 'oz'], 4], [['1', 'oz'], 5], [['3', 'oz'], 6], [['1', 'oz'], 7], [['1', 'cup'], 8]]
[['1/2 oz ', 1], ['1/2 oz ', 2], ['1/2 oz Bacardi ', 3], ['1 oz ', 4], ['1 oz ', 5], ['3 oz ', 6], ['1 oz ', 7], ['1 cup ', 8]]

index:  4
[[['1', 'shot'], 1], [['2', 'shot'], 2], [['1', 'shot'], 3], [['1', 'shot'], 4]]
[['1 shot ', 1], ['2 shots ', 2], ['1 shot ', 3], ['1 shot ', 4]]

index:  5
[[['1/2', 'oz'], 1], [['1/2', 'oz'], 2], [['1/2', 'oz'], 3], [['1/2', 'oz'

In [4]:
UNIT = ["oz", "ounce", "fl oz", "cl", "tsp", "cubes", "tblsp", "tablespoon", "dash", "dashes", "cup", "pinch", "shot", "glass", "glasses", "drop",
        "can", "jigger", "dl", "bottle", "pint", "quart", "qt", "gr", "inch", "splash", "gal", "scoop", "scoops",
        "kg", "lb", "ml", "part", "top up with", "fill to top", "fill", "to fill", "a little bit of", "slice", "juice of",
        "sprig", "sprigs", "wedge", "twist of", "long strip", "whole", "large", "fifth", "piece", "pieces", "chunk", "by taste", "to taste",
        "handful", "stick", "package", "packages", "leaves", "l", "dash of"]

In [5]:
# only one unit will survive
def remove_duplicate_unit(list_):
    for i in range(len(list_)):
        element = list_[i][0]
        count = 0
        remove_index = []
        for index in range(len(element)):
            if element[index] in UNIT:
                count+=1
            if count >= 2:
                remove_index.append(index)
        for del_index in reversed(remove_index):
            element.pop(del_index)
    return list_

all_drinks['Measures'] = all_drinks['Measures'].apply(remove_duplicate_unit)

In [6]:
# verify measure preprocess   # remeber index 74, 85, 103, 163 ...
for i in range(len(all_drinks)):
    print("index: ", i)
    print(all_drinks['Measures'][i])
    print(compared_alcohol['Measures'][i])
    print()

index:  0
[[['1', 'oz'], 1], [['1', 'oz'], 2]]
[['1 oz white ', 1], ['1 oz ', 2]]

index:  1
[[['1/2', 'oz'], 1], [['1/4', 'oz'], 2], [['1/4', 'oz'], 3], [['1/4', 'oz'], 4], [['1/4', 'oz'], 5], [['1/4', 'oz'], 6], [['1/2', 'oz'], 7], [['1/4', 'oz'], 8]]
[['1/2 oz ', 1], ['1/4 oz ', 2], ['1/4 oz ', 3], ['1/4 oz ', 4], ['1/4 oz ', 5], ['1/4 oz ', 6], ['1/2 oz ', 7], ['1/4 oz ', 8]]

index:  2
[[['16', 'oz'], 1], [['1.5', 'oz'], 2]]
[['16 oz ', 1], ['1.5 oz ', 2]]

index:  3
[[['1/2', 'oz'], 1], [['1/2', 'oz'], 2], [['1/2', 'oz'], 3], [['1', 'oz'], 4], [['1', 'oz'], 5], [['3', 'oz'], 6], [['1', 'oz'], 7], [['1', 'cup'], 8]]
[['1/2 oz ', 1], ['1/2 oz ', 2], ['1/2 oz Bacardi ', 3], ['1 oz ', 4], ['1 oz ', 5], ['3 oz ', 6], ['1 oz ', 7], ['1 cup ', 8]]

index:  4
[[['1', 'shot'], 1], [['2', 'shot'], 2], [['1', 'shot'], 3], [['1', 'shot'], 4]]
[['1 shot ', 1], ['2 shots ', 2], ['1 shot ', 3], ['1 shot ', 4]]

index:  5
[[['1/2', 'oz'], 1], [['1/2', 'oz'], 2], [['1/2', 'oz'], 3], [['1/2', 'oz'

In [7]:
# create regex filter
filter = ""
for unit in UNIT:
    if (unit == "oz"):
        filter += unit
    else:
        filter += "|"+ unit
filter

'oz|ounce|fl oz|cl|tsp|cubes|tblsp|tablespoon|dash|dashes|cup|pinch|shot|glass|glasses|drop|can|jigger|dl|bottle|pint|quart|qt|gr|inch|splash|gal|scoop|scoops|kg|lb|ml|part|top up with|fill to top|fill|to fill|a little bit of|slice|juice of|sprig|sprigs|wedge|twist of|long strip|whole|large|fifth|piece|pieces|chunk|by taste|to taste|handful|stick|package|packages|leaves|l|dash of'

#### Create Rule-Based Algorithm

Rule-Based 기반의 알고리즘을 통해, 단위만 분류될 수 있도록 합니다.

여기서 생성된 vis는 아래의 추가적인 전처리 함수를 위해 사용될 것입니다.

vis 결과를 추가적으로 전처리해도, 좋은 결과를 만들 수 있습니다. 그러나 juice of 처럼 구 단위로 되어 있는 경우

띄어쓰기 구분에서 의하여 각각 따로 분류 되기 때문에, 추가적인 규칙을 정의하여야 합니다.

In [8]:
number = ['1', '2', '3', '4', '5', '6', '7', '8', '9', ' ', '/']  # empty space or / for the case of number/unit differentiation
character = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

vis = []
for i in range(len(compared_alcohol)):
    vis.append([])
    for j in range(len(compared_alcohol['Measures'][i])):
        text = str(compared_alcohol['Measures'][i][j][0]).lower() # lower cased
        text = re.sub(',', '', text)
        text = re.sub('\\r|\\n', '', text)

        unit_index = re.search(filter, text) # index_start ~ index_end-1 까지가 포함되는 단위이다. 즉 [start_index : end_index] 이렇게 사용할 수 있다.
        update = True


        if unit_index: # in case of not NoneType -> matched at least one
            start_index, end_index = unit_index.start(), unit_index.end()
            if (start_index == 0) and (end_index == len(text)): # 단위만 나오는 경우
                pass
            elif (start_index != 0) and (end_index == len(text)): # 패턴에 끝에 매칭되는 경우
                if text[start_index-1] in character:    # 단어의 일부로 매칭된 경우 (unit이 아닌 경우)
                    update=False
                elif text[start_index-1] in number: # 공백이나 숫자로 매칭되는 경우
                    pass # 공백이나 숫자로 매칭되는 경우 -> 그 자체로 unit 이거나, 숫자와 붙어있는 유닛인 경우
            elif (start_index == 0) and (end_index != len(text)): # 패턴 시작에 매칭되는 경우
                if text[end_index] in character:
                    update=False # 단어의 일부로 매칭된 경우로 볼 수 있다..
                elif text[end_index] in number:
                    pass # 끝단에 공백이나, 숫자가 등장하는 것은 유닛이 문자바로 앞에 등장하고 뒤에 공백이나 숫자가 바로 따라 붙는 경우를 의미한다.
            elif (start_index != 0) and (end_index != len(text)): #중간에 매칭된 경우
                if (text[start_index-1] in number) and (text[end_index] in number): # 둘다 숫자나 공백으로 둘러 쌓인 경우에만 유닛으로 볼 수 있다.
                    pass
                else:
                    if text[end_index] == "s" or text[end_index:end_index+2] == "es":
                        pass
                    else:
                        update=False
                # 다른 경우는 모두 문자의 일부로 매칭된 경우로 유닛으로 볼 수 없다.

            if update:
                unit = text[start_index:end_index]
                # reconstruct text
                text = text[:start_index] + " " + unit + text[end_index:]

                compared_alcohol['Measures'][i][j][0] = text
            vis[i].append(re.split(' ', text))

        else: #  in case that nothing matches
            vis[i].append(re.split(' ', text)) # 숫자만 존재하는 경우도 있으므로 넣도록 합니다.

# remove '' in the list
for i in range(len(vis)): # by row
    for j in range(len(vis[i])): # by ingredients
        remove_index = []
        for k in range(len(vis[i][j])): # by tokens
            if len(vis[i][j][k]) == 0:
                remove_index.append(k)

        for del_index in reversed(remove_index):
            a = vis[i][j].pop(del_index)

for i in range(len(vis)):
    remove_index = []
    for j in range(len(vis[i])):
        if (len(vis[i][j]) == 0):
            remove_index.append(j)
    for del_index in reversed(remove_index):
        vis[i].pop(del_index)

for i in range(len(vis)):
    print("index: ", i)
    print(vis[i])

index:  0
[['1', 'oz', 'white'], ['1', 'oz']]
index:  1
[['1/2', 'oz'], ['1/4', 'oz'], ['1/4', 'oz'], ['1/4', 'oz'], ['1/4', 'oz'], ['1/4', 'oz'], ['1/2', 'oz'], ['1/4', 'oz']]
index:  2
[['16', 'oz'], ['1.5', 'oz']]
index:  3
[['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz', 'bacardi'], ['1', 'oz'], ['1', 'oz'], ['3', 'oz'], ['1', 'oz'], ['1', 'cup']]
index:  4
[['1', 'shot'], ['2', 'shots'], ['1', 'shot'], ['1', 'shot']]
index:  5
[['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz', 'bacardi']]
index:  6
[['1/2', 'shot', 'bacardi'], ['1/2', 'shot']]
index:  7
[['1/3', 'oz'], ['1/3', 'oz'], ['1/3', 'oz']]
index:  8
[['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz'], ['1-2', 'dash'], ['1', 'wedge']]
index:  9
[['2-3', 'oz']]
index:  10
[['2', '1/2', 'oz'], ['1', 'splash'], ['fill', 'with']]
index:  11
[]
index:  12
[['2', 'oz', 'dry'], ['4', 'oz'], ['0.75', 'oz']]
index:  13
[['1/3', 'part'], ['1/3', 'part'], ['1/3', 'part']]
index:  14
[['2', 'oz'], ['1/2', '

In [9]:
# 길이 검증

# for i in range(len(all_drinks)):
#     if len(vis[i]) != len(all_drinks['Measures'][i]):
#         print("different length!")
#         print("index: ", i)
#         print("len vis: ", len(vis[i]))
#         print("len all_drinks: ",len(all_drinks["Measures"][i]))
#         print()

In [10]:
# 위의 remove_duplicate_unit을 적용한 후로
# 단위는 모두 1개씩만 존재하게 된다. 그러나 단위가 애초에 없는 경우에도 'l', 'l' 이렇게 분류된 경우도 존재하기 때문에,
# 이 경우에도 대해서도 처리가 필요하다.
# 가능한 케이스는  -> 1. 'chilled' -> 'l', 'l', -> 'l'
#               -> 2. '0.5l' -> '0.5', 'l'
#               -> 3. 'l' -> 'l'
#               -> 4. '1.5 chilled' -> '1.5', 'l'
# 이중 숫자 단위, 단위만 존재한다면 이런 경우는 자동으로 해결된다.
# 문제는 관련 없는 문자만 등장하거나, 숫자 문자 이렇게 등장하는 경우이다.

def is_included(token, list_):
    if token in list_:
        return True
    if token + 's' in list_:
        return True
    if token + 'es' in list_:
        return True
    return False


# 구로 표현된 경우는 공백 분리로 인해 일치하는게 없을 것임으로 이런 것들은 따로 고려해줄 필요가 있다. 또한 구로 표현된 경우는 문자로 부터 분리된 경우를 걱정할 필요가 없다.
phrased = ["fl oz", "top up with", "fill to top", "to fill", "a little bit of", "juice of", "twist of", "long strip", "inch strip", "by taste", "to taste"]

for row in range(len(all_drinks)):
    list_ = all_drinks['Measures'][row]
    for i in range(len(list_)):
        element = list_[i][0]
        remove_index = [] # 각 element에 대해 삭제 index를 수집한다.
        for j in range(len(element)):
            tokens = element[j]
            if (tokens in UNIT) and (tokens not in phrased): # 각 행마다 유니크하게 존재하는 단위 (구로 표현된 단위는 제외)
                if is_included(tokens, vis[row][i]): # vis[row][each ingredient]
                    continue # -> 유닛의 단위가 존재하는 경우
                else:
                    remove_index.append(j) # -> 유닛의 단위가 존재하지 않아, 다른 단어로 부터 추출된 경우로 볼 수 있다.

        for del_index in reversed(remove_index):
            all_drinks['Measures'][row][i][0].pop(del_index)

#### Soultion 2

숫자 기반으로 매칭된 end_index 이후 부분은 포함시키지 않습니다.

단위로 매칭된 이후 부분은 단위가 아닌 문자에서 매칭된 것이 아니라면은 제거 대상이기 때문입니다.

In [11]:
number = ['1', '2', '3', '4', '5', '6', '7', '8', '9', ' ', '/']  # empty space or / for the case of number/unit differentiation
character = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

vis = []
for i in range(len(compared_alcohol)):
    vis.append([])
    for j in range(len(compared_alcohol['Measures'][i])):
        text = str(compared_alcohol['Measures'][i][j][0]).lower() # lower cased
        text = re.sub(',', '', text)
        text = re.sub('\\r|\\n', '', text)

        unit_index = re.search(filter, text) # index_start ~ index_end-1 까지가 포함되는 단위이다. 즉 [start_index : end_index] 이렇게 사용할 수 있다.

        update = True

        if unit_index: # in case of not NoneType -> matched at least one
            start_index, end_index = unit_index.start(), unit_index.end()
            if (start_index == 0) and (end_index == len(text)): # 단위만 나오는 경우
                pass
            elif (start_index != 0) and (end_index == len(text)): # 패턴에 끝에 매칭되는 경우
                if text[start_index-1] in character:    # 단어의 일부로 매칭된 경우 (unit이 아닌 경우)
                    update = False
                elif text[start_index-1] in number: # 공백이나 숫자로 매칭되는 경우
                    pass # 공백이나 숫자로 매칭되는 경우 -> 그 자체로 unit 이거나, 숫자와 붙어있는 유닛인 경우
            elif (start_index == 0) and (end_index != len(text)): # 패턴 시작에 매칭되는 경우
                if text[end_index] in character:
                    update = False # 단어의 일부로 매칭된 경우로 볼 수 있다..
                elif text[end_index] in number:
                    pass # 끝단에 공백이나, 숫자가 등장하는 것은 유닛이 문자바로 앞에 등장하고 뒤에 공백이나 숫자가 바로 따라 붙는 경우를 의미한다.
            elif (start_index != 0) and (end_index != len(text)): #중간에 매칭된 경우
                if (text[start_index-1] in number) and (text[end_index] in number): # 둘다 숫자나 공백으로 둘러 쌓인 경우에만 유닛으로 볼 수 있다.
                    pass
                else:
                    if (text[end_index] == "s") or (text[end_index:end_index+2] == "es"): # 복수형을 카운트하기 위한 경우
                        pass
                    else:
                        update = False
                # 다른 경우는 모두 문자의 일부로 매칭된 경우로 유닛으로 볼 수 없다.

            if update:
                unit = text[start_index:end_index]
                # reconstruct text
                text = text[:start_index] + " " + unit # text[end_index:] 이거는 필요가 없습니다.. 어차피 단위 이후 부분은 필요 없는 정보이기 때문입니다.

            vis[i].append(re.split(' ', text))

        else: #  in case that nothing matches
            vis[i].append(re.split(' ', text)) # 숫자만 존재하는 경우도 있으므로 넣도록 합니다.

# remove '' in the list
for i in range(len(vis)): # by row
    for j in range(len(vis[i])): # by ingredients
        remove_index = []
        for k in range(len(vis[i][j])): # by tokens
            if len(vis[i][j][k]) == 0:
                remove_index.append(k)

        for del_index in reversed(remove_index):
            a = vis[i][j].pop(del_index)

for i in range(len(vis)):
    remove_index = []
    for j in range(len(vis[i])):
        if (len(vis[i][j]) == 0):
            remove_index.append(j)
    for del_index in reversed(remove_index):
        vis[i].pop(del_index)

vis

[[['1', 'oz'], ['1', 'oz']],
 [['1/2', 'oz'],
  ['1/4', 'oz'],
  ['1/4', 'oz'],
  ['1/4', 'oz'],
  ['1/4', 'oz'],
  ['1/4', 'oz'],
  ['1/2', 'oz'],
  ['1/4', 'oz']],
 [['16', 'oz'], ['1.5', 'oz']],
 [['1/2', 'oz'],
  ['1/2', 'oz'],
  ['1/2', 'oz'],
  ['1', 'oz'],
  ['1', 'oz'],
  ['3', 'oz'],
  ['1', 'oz'],
  ['1', 'cup']],
 [['1', 'shot'], ['2', 'shot'], ['1', 'shot'], ['1', 'shot']],
 [['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz'], ['1/2', 'oz']],
 [['1/2', 'shot'], ['1/2', 'shot']],
 [['1/3', 'oz'], ['1/3', 'oz'], ['1/3', 'oz']],
 [['1/2', 'oz'],
  ['1/2', 'oz'],
  ['1/2', 'oz'],
  ['1/2', 'oz'],
  ['1/2', 'oz'],
  ['1-2', 'dash'],
  ['1', 'wedge']],
 [['2-3', 'oz']],
 [['2', '1/2', 'oz'], ['1', 'splash'], ['fill']],
 [],
 [['2', 'oz'], ['4', 'oz'], ['0.75', 'oz']],
 [['1/3', 'part'], ['1/3', 'part'], ['1/3', 'part']],
 [['2', 'oz'], ['1/2', 'oz'], ['1', 'splash'], ['1', 'oz']],
 [['1', 'oz'], ['1/2', 'oz'], ['4', 'oz'], ['1/2', 'oz']],
 [['2', 'oz'], ['4', 'oz'], ['1', 'twist', 'of']

In [12]:
phrased = ["fl oz", "top up with", "fill to top", "to fill", "a little bit of", "juice of", "twist of", "long strip", "by taste", "to taste"]

def is_num_included(str):
    number = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    for num in number:
        if num in str:
            return True
    return False

# combine phrased representation
start_phrase = ["fl", "top", "fill", "to", "a", "juice", "twist", "long", "by", "to"] # inch의 경우 독자 단위 임으로, combine하지 않는다.
for i in range(len(vis)):
    for j in range(len(vis[i])):
        index = len(vis[i][j])
        for k in range(len(vis[i][j])):
            if vis[i][j][k] in start_phrase:
                index = k
                break
        st = ""
        for combine in range(index, len(vis[i][j])):
            if combine == index:
                st += vis[i][j][combine]
            else:
                st += " " + vis[i][j][combine]
        for del_index in range(len(vis[i][j])-1, index-1, -1):
            vis[i][j].pop(del_index)

        if len(st) != 0:
            vis[i][j].append(st)

        remove_index = []
        for k in range(len(vis[i][j])):
            if (vis[i][j][k] not in UNIT) and not(is_num_included(vis[i][j][k])):
                remove_index.append(k)
        for del_index in reversed(remove_index):
            vis[i][j].pop(del_index)

In [13]:
for i in range(len(vis)):
    if len(vis[i]) != len(all_drinks['Measures'][i]):
        print("different length!")
        print("index: ", i)
        print("len vis: ", len(vis[i]))
        print("len all_drinks: ",len(all_drinks["Measures"][i]))

In [14]:
# 솔루션 검증
count = 0

for i in range(len(all_drinks)):
    for j in range(len(all_drinks["Measures"][i])):
        if (all_drinks["Measures"][i][j][0] != vis[i][j]):
            count += 1
            print("index: ", i)
            print("all_drinks: ", all_drinks["Measures"][i][j][0])
            print("vis: ", vis[i][j])
            print("compared_alcohol: ", compared_alcohol["Measures"][i][j][0])
            print()

print(count)

index:  21
all_drinks:  ['juice of', '1/2']
vis:  ['juice of']
compared_alcohol:   juice of 1/2 

index:  29
all_drinks:  ['juice of', '1/4']
vis:  ['juice of']
compared_alcohol:   juice of 1/4 

index:  54
all_drinks:  ['juice of', '1']
vis:  ['juice of']
compared_alcohol:   juice of 1 

index:  126
all_drinks:  ['juice of', '1/2']
vis:  ['juice of']
compared_alcohol:   juice of 1/2 

index:  158
all_drinks:  ['juice of', '1/2']
vis:  ['juice of']
compared_alcohol:   juice of 1/2 

index:  159
all_drinks:  ['juice of', '1/2']
vis:  ['juice of']
compared_alcohol:   juice of 1/2 

index:  163
all_drinks:  ['1']
vis:  ['1', 'bottle']
compared_alcohol:  1 small bottle 

index:  170
all_drinks:  ['juice of', '1/2']
vis:  ['juice of']
compared_alcohol:   juice of 1/2 

index:  177
all_drinks:  []
vis:  ['glass']
compared_alcohol:  full glass 

index:  181
all_drinks:  ['juice of', '1']
vis:  ['juice of']
compared_alcohol:   juice of 1 

index:  181
all_drinks:  ['juice of', '1']
vis:  ['jui

In [15]:
# 발견된 문제점들


#index:  395
# all_drinks:  ['3', '2']
# vis:  ['3', '2-inch']
# compared_alcohol:  3 2-inch strips
# 설명 :  3 2-inch strips의 경우 '3', '2', 'inch'이렇게 잘리는데, 최종 vis와의 비교에서 2-inch는 존재하지 않기 때문에 사라지게 된다.


# index:  222
# all_drinks:  ['70', '2']
# vis:  ['70ml/2fl', 'oz']
# compared_alcohol:  70ml/2fl oz
# 설명 : 70ml/2fl oz의 경우 ['70', 'ml', '2', 'fl oz']로 잘리는데, 여기서 중복된 단위는 삭제
#와 vis와의 비교를거쳐 단위가 모두 짤리게 된다..


# index:  177
# all_drinks:  []
# vis:  ['glass']
# compared_alcohol:  full glass
# 설명 : full glass와 같은 경우, full -> 'l', 'l', 'glass', 이렇게 분류 되는데 여기서, 중복된 단어 삭제를 거쳐 ->
# 'ㅣ'만 남게되고, -> 'l'은 vis에는 존재하지 않음으로 삭제되어 빈칸만 남게된 것이다.

# index:  163
# all_drinks:  ['1']
# vis:  ['1', 'bottle']
# compared_alcohol:  1 small bottle
# 설명 : 1 small bottle또한 '1', 'l', 'l', 'bottle' -> '1', 'l' -> '1'만 남게된 것이다.


In [16]:
all_drinks['Measures'][395][2][0] = ['3', '2', 'inch']
all_drinks['Measures'][163][4][0] = ['1', 'bottle']
all_drinks['Measures'][177][2][0] = ['glass']
all_drinks['Measures'][222][0][0] = ['70ml', '2fl oz']

In [17]:
all_drinks.to_pickle('./preprocessed_alcohol_v2.pkl')

In [18]:
all_drinks['Measures'][0]

[[['1', 'oz'], 1], [['1', 'oz'], 2]]

#### Ingredients와의 정합성 검증

In [19]:
for i in range(len(all_drinks)):
    ingredients = all_drinks['Ingredients'][i]
    print("index :", i)
    print("Measures: ", all_drinks['Measures'][i])
    for j in range(len(ingredients)):
        print(ingredients[j])

index : 0
Measures:  [[['1', 'oz'], 1], [['1', 'oz'], 2]]
['creme de cacao', 1]
['vodka', 2]
index : 1
Measures:  [[['1/2', 'oz'], 1], [['1/4', 'oz'], 2], [['1/4', 'oz'], 3], [['1/4', 'oz'], 4], [['1/4', 'oz'], 5], [['1/4', 'oz'], 6], [['1/2', 'oz'], 7], [['1/4', 'oz'], 8]]
['absolut kurant', 1]
['grand marnier', 2]
['chambord raspberry liqueur', 3]
['midori melon liqueur', 4]
['malibu rum', 5]
['amaretto', 6]
['cranberry juice', 7]
['pineapple juice', 8]
index : 2
Measures:  [[['16', 'oz'], 1], [['1.5', 'oz'], 2]]
['lager', 1]
['tequila', 2]
index : 3
Measures:  [[['1/2', 'oz'], 1], [['1/2', 'oz'], 2], [['1/2', 'oz'], 3], [['1', 'oz'], 4], [['1', 'oz'], 5], [['3', 'oz'], 6], [['1', 'oz'], 7], [['1', 'cup'], 8]]
['malibu rum', 1]
['light rum', 2]
['151 proof rum', 3]
['dark creme de cacao', 4]
['cointreau', 5]
['milk', 6]
['coconut liqueur', 7]
['vanilla ice-cream', 8]
index : 4
Measures:  [[['1', 'shot'], 1], [['2', 'shot'], 2], [['1', 'shot'], 3], [['1', 'shot'], 4]]
['dark rum', 1]


In [20]:
len(all_drinks['Ingredients'][0])
len(all_drinks['Measures'][0])

count = 0

for i in range(len(all_drinks)):
    if (len(all_drinks['Ingredients'][i]) != len(all_drinks['Measures'][i])):
        count+=1
        print("different length between Ingredients and Measures!")
        print("index: ", i)
        print("Drink Name: ", all_drinks['DrinkName'][i])
        print("number of Ingredients: ", len(all_drinks['Ingredients'][i]))
        print("number of Measures: ", len(all_drinks['Measures'][i]))
        print()

print("total count : ", count)

different length between Ingredients and Measures!
index:  8
Drink Name:  3-Mile Long Island Iced Tea
number of Ingredients:  9
number of Measures:  7

different length between Ingredients and Measures!
index:  9
Drink Name:  410 Gone
number of Ingredients:  2
number of Measures:  1

different length between Ingredients and Measures!
index:  11
Drink Name:  501 Blue
number of Ingredients:  5
number of Measures:  0

different length between Ingredients and Measures!
index:  19
Drink Name:  A Piece of Ass
number of Ingredients:  4
number of Measures:  3

different length between Ingredients and Measures!
index:  21
Drink Name:  A True Amaretto Sour
number of Ingredients:  4
number of Measures:  2

different length between Ingredients and Measures!
index:  22
Drink Name:  A midsummernight dream
number of Ingredients:  5
number of Measures:  4

different length between Ingredients and Measures!
index:  33
Drink Name:  Absolut Evergreen
number of Ingredients:  4
number of Measures:  3

diff

In [21]:
# 원래부터, 일부 Measure 값은 누락되어 있기 때문에, 모두 반영되어 있지 않는다.
print(all_drinks['DrinkName'][11])
print(all_drinks['Ingredients'][11])
print(all_drinks['Measures'][11])
print(compared_alcohol['Measures'][11]) # original alcohol information.

501 Blue
[['blue curacao', 1], ['blueberry schnapps', 2], ['vodka', 3], ['sour mix', 4], ['7-up', 5]]
[]
[]


In [22]:
for i in range(len(all_drinks)):
    print("index: ", i)
    print("ALL DrinK: ", all_drinks['Measures'][i])
    print("Compared Alcohol: ", compared_alcohol['Measures'][i])
    print("Ingredients: ", all_drinks['Ingredients'][i])
    if (len(all_drinks['Ingredients'][i]) != len(all_drinks['Measures'][i])):
        print("number of Ingredients: ", len(all_drinks['Ingredients'][i]))
        print("number of Measures: ", len(all_drinks['Measures'][i]))
    print()

index:  0
ALL DrinK:  [[['1', 'oz'], 1], [['1', 'oz'], 2]]
Compared Alcohol:  [['1  oz white ', 1], ['1  oz ', 2]]
Ingredients:  [['creme de cacao', 1], ['vodka', 2]]

index:  1
ALL DrinK:  [[['1/2', 'oz'], 1], [['1/4', 'oz'], 2], [['1/4', 'oz'], 3], [['1/4', 'oz'], 4], [['1/4', 'oz'], 5], [['1/4', 'oz'], 6], [['1/2', 'oz'], 7], [['1/4', 'oz'], 8]]
Compared Alcohol:  [['1/2  oz ', 1], ['1/4  oz ', 2], ['1/4  oz ', 3], ['1/4  oz ', 4], ['1/4  oz ', 5], ['1/4  oz ', 6], ['1/2  oz ', 7], ['1/4  oz ', 8]]
Ingredients:  [['absolut kurant', 1], ['grand marnier', 2], ['chambord raspberry liqueur', 3], ['midori melon liqueur', 4], ['malibu rum', 5], ['amaretto', 6], ['cranberry juice', 7], ['pineapple juice', 8]]

index:  2
ALL DrinK:  [[['16', 'oz'], 1], [['1.5', 'oz'], 2]]
Compared Alcohol:  [['16  oz ', 1], ['1.5  oz ', 2]]
Ingredients:  [['lager', 1], ['tequila', 2]]

index:  3
ALL DrinK:  [[['1/2', 'oz'], 1], [['1/2', 'oz'], 2], [['1/2', 'oz'], 3], [['1', 'oz'], 4], [['1', 'oz'], 5], [['3

In [23]:
unique_glass_name = np.unique(all_drinks['Glass'])
print(unique_glass_name)

['Beer Glass' 'Beer mug' 'Beer pilsner' 'Brandy snifter' 'Champagne Flute'
 'Champagne flute' 'Cocktail Glass' 'Cocktail glass' 'Coffee Mug'
 'Coffee mug' 'Collins Glass' 'Collins glass' 'Copper Mug' 'Cordial glass'
 'Highball Glass' 'Highball glass' 'Hurricane glass' 'Irish coffee cup'
 'Jar' 'Margarita glass' 'Margarita/Coupette glass' 'Martini Glass'
 'Mason jar' 'Old-Fashioned glass' 'Old-fashioned glass' 'Parfait glass'
 'Pint glass' 'Pitcher' 'Pousse cafe glass' 'Punch Bowl' 'Punch bowl'
 'Shot Glass' 'Shot glass' 'Whiskey sour glass' 'White wine glass'
 'Wine Glass']


In [24]:
##################
# Glass 용량 정보  #
##################

# glass 용량에 따라 양이 정해지는 경우를 위해, 미리 mapper를 만듭니다.
# measure에서 "fill"로 나타나면 사용되어야 합니다.

# Beer Glass : 570ml
# Beer mug : 330~500ml (330, 320, 500)
# Beer pilsner : 200~400ml (200, 250, 300, 330, 400)
# Brandy snifter : 180~240ml
# Champagne flute : 180~300ml
# Cocktail Glass : 90~300ml
# Coffee Mug : 240~350ml
# Collins Glass : 300~410ml
# Copper Mug : 200~300ml
# Cordial glass : 44.36ml
# Highball Glass : 240~350ml
# Hurricane glass : 590ml
# Irish coffee cup : 250ml
# Jar : depend on size -> 250~400, 500~700ml
# Margarita glass : 250ml
# Coupette glass : 150~200ml
# Martini Glass : 150 ~ 235ml
# Mason jar : 500ml
# old-fashioned glass : 180~300ml
# parfait glass : 150~350ml
# pint glass : 470~570ml
# pitcher glass : 1780ml
# pousse cafe glass : 120~180ml
# punch bowl : 4000ml
# Shot glass : 35ml
# Whiskey sour glass:207ml
# White wine glass: 150~300ml
# wine glass : 150ml


In [25]:
def print_drink_name(index):
    print(all_drinks['DrinkName'][index])
print_drink_name(434)

def print_drink_glass(index):
    print(all_drinks['Glass'][index])
print_drink_glass(434)

Royal Gin Fizz
Highball glass


In [26]:
##############
# 직접 수정하기 #
##############

#####################################################################################################
# '-'는 'to'를 의미합니다. ex) 1-2는 1에서 2를 의미합니다. 실제 정수로 표현하기 위해 따로 값을 정하는 알고리즘을 만드시오.
# '3/2'와 같이 분수로 표현된 경우 바로 float()으로 변환이 불가능 합니다. 해당 값을 변환시킬 수 있는 알고리즘을 개발하십시오.
# "fill", "to fill"와 같이 glass의 용량에 따라 ml로 변환할 수 있는 알고리즘을 개발하십시오. 다만 과일같은 경우 용량에 합의가 필요합니다.
# index 26과 같이 숫자만 나와 있으면서 단위가 없는 경우는 전체 값에 대한 부분 의미입니다. glass 용량에 따라 부피로 변환하는 알고리즘을 개발하십시오.
# '1', '1/2'와 같이 정수+숫자로 나열된 경우 '3/2' 이렇게 표현되어야 합니다. 해당 값으로 표현하도록 값을 수정하는 알고리즘을 개발하십시오.
# "fill to top"은 나머지 glass의 용량을 채우라는 것입니다. 해당 값을 표현할 수 있도록 알고리즘을 개발하십시오..
# "part"는 자체 재료에 따라 달라집니다. 합의를 보고 이를 고려하여 정량값으로 표현할 수 있도록 알고리즘을 개발하십시오.. (index 80과 같은 경우를 보십시오.)
# garnish는 용량이 정확하지 않습니다. 따로 합의를 보십시오. (숫자와 함께 표현되어 있습니다.)
# 얼음 cube 참가는 량이 정해져 있지 않습니다. 따로 합의를 보십시오. (숫자와 함께 표현되어 있는 경우도 있습니다.) 또한 cubes로 각설탕이 있는 경우도 있습니다. 따로 구분해서 처리해야합니다.
# juice of는 과일마다 착즙량이 다릅니다. 따로 합의를 보십시오.
# twist of는 레몬이나 오렌지 껍질을 나타내는데 사용됬습니다. 따로 합의를 보십시오. (숫자가 왼쪽에 등장하는 경우도 있습니다.)
# top up with 용량이 일정하지 않습니다. 따로 합의를 보십시오.
# leaves는 garnish와 비슷합니다만, 허브이기 때문에 애당시초에 양이 적고 영향이 커서 다르게 구분했습니다.
# by taste와 같이 개인 취향에 따라 달라지는 경우는 합의를 보십시오.
# a little bit of는 용량이 일정하지 않습니다. 따로 합의를 보십시오.
# glass 용량이 일정하지 않습니다. 정해진 하나의 값으로 합의를 보십시오.
# (Collins Glass=350ml로 가정)
#####################################################################################################

# garnish -> garnish는 장식용으로 사용되는 재료입니다. 위의 UNIT에는 포함되지 않으며, 장식임을 나타내기 위해 따로 정의하였습니다. 아주 적은량으로 나타내도 괜찮습니다.
# top up with -> 만들어진 술 위에 조금 추가하는 수준입니다. 계산상 20~30ml도 존재했습니다.
# fill to top -> 남은 glass 용량까지 채우라는 것입니다.
# wedge -> 단위 불명확 (일정한 양이 정해져 있지 않습니다..)
# cubes -> 얼음 큐브의 큐브입니다. 얼음 큐브는 취향에 따라 달라질 수 있고, 아니면 glass의 용량에 따라 고려하도록 합니다. (!cube가 아닌 cubes로 나타내야함!) 또한 각 설탕이 등장하는 경우도 있습니다.
# fill -> glass 용량을 참조하여 채웁니다.
# part -> 왼쪽에 값이 나타납니다. 재료에 따라 부피가 다릅니다. (ex 1/2 part) 또는 glass 전체 용량 중에 일부로 해석할 수 있습니다.
# juice of -> 오른쪽에 값이 나타납니다. 과일마다 얻을 수 있는 용량이 다릅니다. (ex juice of 1/2)
# twist of -> 레몬 껍질, 오렌지 껍질을 표현하는데 사용된 단위입니다. 양을 정하기 애매합니다.
# gr -> gram을 의미합니다.
# leaves -> leave가 아닌 복수형으로 적었습니다. 참고 바랍니다..
# by taste -> 개인 취향에 따라 달라지는 경우입니다.
# a little bit of -> 작은 양을 표현하는데 사용했습니다.
# whole -> 갯수를 의미하는데, 재료에 따라 부피나 그램이 다릅니다. (clove, lemon, orange 등이 포함되어 있습니다.)

def measure_append(row, measure_list, ingredient_index, description=None):
    all_drinks['Measures'][row].append([measure_list, ingredient_index])

def measure_modify(row, measure_list, measure_index, description=None):
    all_drinks['Measures'][row][measure_index][0] = measure_list


measure_append(8, ['1/2', 'oz'], 6, description="빠진 coca-cola를 추가했습니다.")
measure_append(8, ['1', 'garnish'], 9, description='lemon 값의 구체적인 양이 없어서, chunk로 대체했습니다. 장식용입니다.')

measure_append(9, ['1', 'can'], 2, description='coca-cola 표시가 없어서, 1 can으로 대체했습니다.')

measure_modify(10, ['5/2', 'oz'], 0, description="2 1/2 = 5/2 이기 때문에, 사용하기 편하기 위해 수정했습니다.")

measure_append(11, ['70', 'ml'], 1, description="blue curacao, instruction에 따라 Collins Glass/5 만큼 추가했습니다.")
measure_append(11, ['70', 'ml'], 2, description='blueberry schnapps, 위와 동일합니다.')
measure_append(11, ['70', 'ml'], 3, description='vodka, 위와 동일합니다.')
measure_append(11, ['70', 'ml'], 4, description='sour mix, 위와 동일합니다.')
measure_append(11, ['70', 'ml'], 5, description='7-up soda, 위와 동일합니다. Collins Glass=350ml으로 가정했습니다.')

measure_append(19, ["fill"], 4, description="sour mix는 전체 다 채우라고 나와있습니다. fill은 위의 glass 용량 정보를 참고하여 한번에 변환하도록 합니다.")

measure_append(21, ['cubes'], 3, description="Ice는 Cube 단위로 다루기로 합니다. glass에 따라 추후에 얼마나 들어갈지 mapping을 결정할 수 있습니다.")
measure_append(21, ['1', 'garnish'], 4, description="maraschino cherry는 장식용입니다. 그냥 chunk로 표현하겠습니다.")

measure_modify(22, ['5', 'whole'], 3, description='딸기 5개를 나타내기 위해 whole을 사용했습니다. 해당 단위는 갯수를 의미합니다.')
measure_append(22, ["fill"], 5, description="Schweppes Russchain은 fill up with로 지시되어 있습니다. glass 용량 정보를 참고해서 ml로 변환시키시오.")

measure_modify(23, ['3/2', 'oz'], 0, description='1+1/2를 간단히 나타내기 위해 3/2로 수정함.')

measure_modify(25, ['7/4', 'shot'], 0, description='1 + 3/4 = 7/4로 수정함.')

measure_modify(29, ['3/2', 'oz'], 0, description='1+1/2를 간단히 표현.')
measure_modify(29, ['1', 'whole'], 3, description="cherry 1개를 의미합니다. 사실 garnish와 다름없다 봅니다.")

measure_modify(31, ['3/2', 'oz'], 0, description='1+1/2를 간단히 표현.')

measure_modify(32, ['1', 'whole'], 4, description='wormwood 1개를 whole로 표현.')

measure_append(33, ['top up with'], 4, description='bitter lemon 음료를 위에 살짝 뿌리는 수준입니다. 일반 장식과는 다르지만 정량 측정이 얘매합니다.')

measure_modify(37, ['top up with'], 3, description='tonic water를 위에 살짝 부리십시오..')

measure_modify(38, ['cubes'], 3, description='Ice를 추가하는 것인데, 이것은 위의 cube 아이스를 추가하는 것임으로 cubes로 표현했습니다.')

measure_modify(41, ['1', 'whole'], 4, description='1개의 egg white를 표현하기 위해 whole를 사용했습니다.')
measure_modify(41, ['1', 'leaves'], 5, description='mint leaves 1개를 의미합니다.')

measure_modify(42, ['1/2', 'whole'], 4, description='1/2개의 egg white를 표현했습니다.')

measure_append(45, ['by taste'], 5, description='개인 취향에 따라 fruite 추가')
measure_append(45, ['cubes'], 6, description='Ice cube의 량은 합의를 바랍니다.')
measure_append(45, ['by taste'], 8, description='개인 취향에 따라 Fruit juice 추가')

measure_append(52, ['top up with'], 4, description='club soda를 음료 위에 약간 추가하십시오.')

measure_modify(54, ['1', 'wedge'], 3, description='lime 한 조각을 추가하십시오.')

measure_append(55, ['1/4'], 1, description='instruction을 참조했습니다. 컵의 1/4만큼 부으세요.')
measure_append(55, ['1/2'], 2, description='instruction을 참조했습니다. 컵의 1/2만큼 부으세요.')
measure_append(55, ['top up with'], 3, description='instruction을 참조했습니다. 마지막 위에 추가하십시오.')

measure_append(57, ["fill to top"], 3, description='instruction에 따라 추가했습니다. 컵의 마지막까지 채우세요.')

measure_modify(59, ['3/2', 'part'], 2, description='part 단위는 표시되어 있지 않지만 의미상 part를 사용하는 걸로 유추해 볼 수 있어서, part를 사용했습니다.')
measure_append(59, ["top up with"], 5, description='Cream을 올려 마무리 하세요. 크림은 일반 액체와 다르지만 이렇게 구성하였습니다.')

measure_append(62, ['a little bit of'], 4, description='약간의 nutmeg를 추가하십시오.')

measure_modify(69, ['1', 'whole'], 2, description='egg white 1개를 표시했습니다.')

measure_modify(74, ['2', 'whole'], 2, description='apricot 2개를 표시했습니다.')

measure_modify(75, ['1', 'wedge'], 1, description='lime 조각을 추가하여 마무리하십시오.')

measure_append(76, ['fill to top'], 3, description='club soda로 glass 용량을 가득 채우십시요.')

measure_modify(81, ['1', 'splash'], 3, description='Orange juice 1 splach만큼 추가하십시오.')

measure_append(84, ['1/2', 'part'], 1, description='amaretto 1/2 part만큼 추가하십시오.')
measure_append(84, ['1/2', 'part'], 3, description='melon liqeur 1/2 part만큼 추가하십시오.')
measure_append(84, ['1', 'part'], 4, description='pineapple juice 1 part만큼 추가하십시오.')
measure_append(84, ['top up with'], 2, description='Sweet and Sour로 마무리 하십시오.')

measure_modify(85, ['top up with'], 2, description='chilled whipped cream으로 마무리하세요.')

measure_modify(88, ['1', 'inch'], 3, description='1 inch cinnamon을 추가하십시오.')
measure_modify(88, ['1', 'part'], 2, description='allspice 1 part로 유추해서 적었습니다.')
measure_modify(88, ['3-6', 'whole'], 4, description='crushed anise 3에서 6개로 유추해서 적었습니다.')


measure_modify(89, ['1', 'garnish'], 3, description='maraschino cherry 1개를 추가하십시오.')

measure_modify(90, ['2', 'whole'], 1, description='apple 2개를 추가하십시오.')

measure_modify(98, ['1', 'whole'], 4, description='egg white 1개를 추가하십시오.')
measure_modify(98, ['1', 'wedge'], 5, description='orange 1개 조각을 사용하십시오.')

measure_modify(99, ['4', 'fifth'], 1, description='4개의 샴페인 병(750ml)을 추가하십시오.')
measure_modify(99, ['1', 'fifth'], 2, description="1개의 보드가 병(750ml)을 추가하십시오.")

measure_modify(100, ['1', 'wedge'], 3, description='lime 1개 조각을 사용하십시오.')

measure_modify(101, ['cubes'], 3, description='많은 양의 Ice Cube를 추가하십시오.')

measure_modify(103, ['fill'], 0, description='champagne로 가득 채우십시오.')

measure_modify(106, ['cubes'], 7, description="crushed된 Ice Cube를 사용합니다.")

measure_modify(110, ['1', 'fifth'], 2, description='fifth->750ml 병으로 해석했습니다.')

measure_append(112, ['to taste'], 5, description='취향에 따라 설탕을 추가하십시오.')
measure_append(112, ['cubes'], 6, description="Ice Cube를 추가하십시오.")

measure_append(114, ['fill'], 3, description='glass를 풀로 채우십시오.')

measure_append(116, ['top up with'], 5, description='lemonade를 추가하여 마무리 하십시오.')

measure_append(124, ['top up with'], 3, description='cream을 위에 추가하고 모든 재료를 섞으면 됩니다.')

measure_modify(126, ['1', 'piece'], 1, description='banana 1개를 추가하십시오. (1손 아님)')

measure_modify(127, ['1', 'piece'], 2, description='banana 1개를 추가하십시오. (1손 아님)')
measure_modify(127, ['garnish'], 5, description='cherry 1개를 추가하십시오.')

measure_modify(128, ['1/2', 'wedge'], 3, description='banana 1/2 조각을 추가하십시오.')

measure_modify(129, ['1', 'piece'], 1, description='얼음 바나나 1개를 추가하십시오. (1손 아님)')

measure_modify(130, ['1', 'piece'], 1, description='banana 1개를 추가하십시오. piece로 표현')

measure_modify(131, ['top up with'], 4, description='prosecco로 마무리 하십시오.')

measure_modify(134, ['twist of'], 4, description='lemon peel을 추가하십시오. 1개 단위를 없애고 twist of로 바꾸었습니다.')

measure_append(135, ['fill'], 4, description='carbonated water로 가득채우십시오. (fill to top에 가까운 의미)')
measure_append(135, ['twist of'], 5, description='lemon peel을 추가하십시오. twsit of로 표현했습니다.')

measure_append(141, ['3', 'scoops'], 2, description='choloate syrup 3 scoop를 추가하십시오.')
measure_append(141, ['top up with'], 3, description='instruction을 보아도 정보를 유추할 수 없습니다. 임의로 1/2로 표기했습니다. cherry brandy를 마지막에 추가하십시오.')
measure_append(141, ['1/2'], 4, description='instruction을 보아도 정보를 유추할 수 없습니다. 임의로 1/2로 표기했습니다. vodka 술을 추가하십시오.')
measure_append(141, ['1/4', 'cup'], 5, description='instruction을 보아도 정보를 유추할 수 없습니다. 1/4 cup 만큼 milk를 추가하십시오.')

measure_append(149, ['fill'], 3, description='lemonade를 가득 부으십시오.')
measure_append(149, ['1', 'garnish'], 4, description='cherry를 장식용으로 추가하십시오.')

measure_modify(150, ['a little bit of'], 3, description='굵은 소금을 사용합니다. 구체적인 양이 정해져 있지 않아, a little bit of를 사용했습니다.')

measure_modify(152, ['1', 'garnish'], 4, description='cherry를 장식용으로 추가하십시오. (1개로 표기되어 있으나, 장식의 의미가 가까움으로 garnish로 표현)')
measure_modify(152, ['1', 'twist of'], 5, description='lemon peel을 추가하십시오.')

measure_modify(154, ['1', 'twist of'], 3, description='lemon peel을 추가합니다. 숫자는 표시하지 않았습니다.')

measure_modify(155, ['1', 'garnish'], 4, description='cheery를 장식용으로 추가함')

measure_modify(156, ['15', 'whole'], 6, description='mini-snicker 과자 1개 bar당 1whole로 계산했습니다.')
measure_modify(156, ['top up with'], 3, description='wipped cream으로 마무리 하십시오.')

measure_modify(159, ['1', 'whole'], 3, description='egg white 1개를 추가하십시오.')
measure_modify(159, ['1', 'garnish'], 5, description='cherry를 장식용으로 추가하십시오.')

measure_modify(160, ['twist of'], 4, description='lemon peel을 추가하십시오.')

measure_modify(161, ['1', 'slice'], 3, description='orange를 (1 슬라이스) 장식용으로 추가하십시오.')
measure_modify(161, ['1', 'garnish'], 4, description='maraschino cherry를 장식용으로 추가하십시오.')

measure_modify(162, ['1', 'whole'], 4, description='egg white 1개를 추가하십시오.')

measure_modify(163, ['1', 'fifth'], 0, description='1개 everclear 병(750ml)을 추가하십시오.')
measure_modify(163, ['1', 'fifth'], 1, description='vodka 1개 병(750ml)을 추가하십시오.')

measure_append(167, ['a little bit of'], 4, description='아주 약간은 Nutmeg를 sprinkle하는 방식으로 해석했습니다.')

measure_modify(168, ['1', 'slice'], 2, description='레몬 1 슬라이스를 장식용으로 추가합니다.')
measure_modify(168, ['garnish'], 4, description='cherry를 장식용으로 추가합니다.')
measure_modify(168, ['1', 'slice'], 5, description='lemon 1 슬라이스를 장식용으로 추가합니다.')

measure_modify(170, ['garnish'], 4, description='cherry를 장식용으로 추가합니다.')

measure_append(172, ['1/3'], 1, description='총 glass 용량에 1/3을 추가합니다.')
measure_append(172, ['1/3'], 2, description='총 glass 용량에 1/3을 추가합니다.')
measure_append(172, ['1/3'], 3, description='총 glass 용량에 1/3을 추가합니다.')

measure_modify(175, ['fill'], 0, description='corona 술 1개를 모두 소진하여 glass를 채우도록 합니다.')

measure_modify(177, ['1', 'glass'], 2, description='1 glass 용량으로 숫자를 추가했습니다.')

measure_append(178, ['fill'], 1, description='거의 glass를 coffee로 가득 채우십시오.')
measure_append(178, ['1/2', 'oz'], 4, description='실제 brandy 량이 나타나있지 않습니다. 1/2로 가정합니다.')

measure_modify(179, ['1', 'whole'], 1, description='lime 하나를 소비해서 술에 추가합니다.')

measure_modify(180, ['2', 'whole'], 0, description='2개의 라임을 사용합니다.')
measure_modify(180, ['cubes'], 3, description='crushed된 Ice cube를 사용합니다.')

measure_append(181, ['fill to top'], 6, description='carbonated water로 남은 glass의 용량을 채우십시오.')

measure_modify(186, ['garnish'], 4, description='cherry를 장식용으로 추가합니다.')

measure_modify(187, ['1', 'whole'], 4, description='달걀 노른자 1개를 추가합니다.')

measure_modify(189, ['fill'], 0, description='champagne을 가득 채웁니다.')

measure_modify(192, ['1', 'whole'], 4, description='1개의 달걀을 추가합니다.')
measure_append(192, ['fill'], 6, description='carbonated water로 가득채웁니다.')

measure_modify(193, ['3', 'whole'], 3, description='달걀 3개를 추가합니다.')


measure_append(195, ['2', 'oz'], 3, description='water을 약간 추가합니다. a little bit of는 매우 작은 양을 나타내는데 사용했음으로, 대략적으로 80ml정도 나오는 2oz를 사용했습니다.')

measure_modify(197, ['top up with'], 5, description='whipped cream으로 마무리합니다.')
measure_modify(197, ['garnish'], 6, description='cherry는 장식취급합니다.')

measure_modify(200, ['cubes'], 2, description='각설탕을 집어넣을 필요가 있습니다.')
measure_modify(200, ['1', 'slice'], 4, description='1개의 oragne 슬라이스를 추가합니다.')
measure_modify(200, ['1', 'garnish'], 5, description='cherry를 장식용으로 추가합니다.')

measure_modify(202, ['1', 'whole'], 3, description='egg white 1개를 추가합니다.')

measure_modify(204, ['1', 'sprig'], 3, description='구체적인 vanilla 빈의 단위를 추가합니다.')
measure_append(204, ['1', 'tsp'], 6, description='caramel coloring 제품을 1tsp 만큼 추가합니다.')

measure_append(210, ['fill'], 2, description='ginger ale로 채우면 됩니다.')

measure_modify(212, ['juice of', '1'], 1, description='라임즙을 전부 사용합니다.')
measure_append(212, ['cubes'], 4, description='Ice를 추가합니다.')

measure_append(213, ['fill'], 3, description='coca-cola로 채우면 됩니다.')

measure_modify(217, ['3', 'wedge'], 1, description='lime 3개 조각정도로 유추해서 적었습니다.')

measure_modify(222, ['70', 'ml'], 0, description='vodka 70ml를 추가합니다.')
measure_modify(222, ['1', 'tbsp'],1, description='tbsp단위를 캐치하지 못해서 추가했습니다.')
measure_modify(222, ['2', 'tbsp'],2,  description='tbsp단위를 캐치하지 못해서 추가했습니다.')
measure_modify(222, ['1', 'garnish'], 4, description='olive를 마지막에 추가합니다.')

measure_append(223, ['1/3'], 1, description='instruction에도 정보가 없어 유추해서 적었습니다.')
measure_append(223, ['1/3'], 2, description='instruction에도 정보가 없어 유추해서 적었습니다.')
measure_append(223, ['1/3'], 3, description='instruction에도 정보가 없어 유추해서 적었습니다.')

measure_modify(224, ['top up with'], 3, description='마지막으로 추가하는 재료로 top up with의 의미로 해석했습니다.')

measure_modify(225, ['1', 'wedge'], 2, description='lime 1조각을 추가합니다.')

measure_modify(226, ['1', 'strig'], 3, description='1 Vanilla bean 줄기를 추가하는 걸로 해석했습니다.')
measure_modify(226, ['top up with'], 5, description='whipped cream으로 마무리합니다.')

measure_modify(231, ['6', 'whole'], 0, description='egg yolk 6개를 사용합니다.')
measure_modify(231, ['6', 'whole'], 8, description='6개의 달걀 흰자를 사용합니다.')
measure_modify(231, ['a little bit of'], 10, description='ground nutmeg는 마지막에 약간 사용합니다.')

measure_append(232, ['a little bit of'], 7, description='ground nutmeg는 마지막에 약간 올립니다.')

measure_modify(233, ['6', 'whole'], 0, description='6개의 달걀을 사용합니다.')

measure_append(234, ['fill'], 4, description='carbonated water로 채웁니다.')
measure_append(234, ['1', 'twist of'], 5, description='lemon peel을 추가합니다.')

measure_modify(235, ['1', 'garnish'], 5, description='cherry를 장식용으로 추가합니다.')

measure_modify(246, ['1', 'wedge'], 4, description='orange 조각을 추가합니다.')
measure_modify(246, ['1', 'garnish'], 5, description='cherry를 장식용으로 추가합니다.')

measure_modify(247, ['1', 'wedge'], 4, description='orange 조각을 추가합니다.')
measure_modify(247, ['1', 'garnish'], 5, description='cherry를 장식용으로 추가합니다.')

measure_modify(251, ['1', 'garnish'], 4, description='cheery를 장식용으로 추가합니다.')

measure_modify(252, ['6', 'leaves'], 2, description='6개의 mint 잎을 추가합니다.')

measure_modify(254, ['2', 'slice'], 3, description='2개의 lemon 슬라이스를 추가합니다.')
measure_modify(254, ['1', 'slice'], 4, description='1개의 사과 슬라이스를 추가합니다.')
measure_append(254, ['cubes'], 7, description='ice cube를 추가합니다.')


measure_modify(256, ['1', 'piece'], 1, description='바나나 1조각을 추가합니다.')
measure_modify(256, ['6', 'cubes'], 4, description='6개의 얼음 큐브를 추가합니다.')

measure_append(258, ['fill to top'], 2, description='ginger ale을 마지막까지 채웁니다.')
measure_append(258, ['cubes'], 3, description='ice cube를 추가합니다.')

measure_modify(262, ['1', 'wedge'], 2, description='라임 1조각을 추가합니다.')

measure_append(263, ['fill'], 2, description='carbonated water로 채웁니다.')
measure_append(263, ['to taste'], 3, description='설탕은 따로 구체적인 양이 정해져있지 않음으로, 취향에 따라 설정하도록 하였습니다.')
measure_append(263, ['1', 'wedge'], 4, description='orange spiral을 추가합니다.')
measure_append(263, ['twist of'], 5, description='lemon peel을 추가합니다.')

measure_modify(264, ['1', 'garnish'], 4, description='cherry를 장식용으로 추가합니다.')
measure_modify(264, ['1', 'wedge'], 5, description='orange 조각을 추가합니다.')

measure_append(265, ['fill to top'], 4, description='carbonated water를 마지막 까지 채웁니다.')

measure_modify(266, ['1', 'wedge'], 4, description='lime 조각을 추가합니다.')

measure_modify(268, ['1', 'cubes'], 2, description='설탕 큐브를 의미합니다.')
measure_modify(268, ['4', 'leaves'], 3, description='4개의 mint 잎을 추가합니다.')
measure_modify(268, ['1', 'garnish'], 5, description='cherry를 장식용으로 추가합니다.')

measure_modify(269, ['1', 'wedge'], 3, description='orange 조각을 추가합니다.')
measure_modify(269, ['1', 'garnish'], 4, description='cherry를 장식용으로 추가합니다.')

measure_modify(270, ['2', 'whole'], 4, description = '딸기 2개 추가')
measure_append(270, ['fill'], 6, description='carbonated water로 glass를 채웁니다.')


measure_modify(274, ['a little bit of'], 3, description='cinnamon을 추가합니다.')
measure_modify(274, ['3', 'whole'], 4, description='3개의 clove를 추가합니다.')

measure_modify(280, ['1/4', 'part'], 1, description='1/4 part lemon를 추가합니다.')
measure_modify(280, ['1/2', 'part'], 2, description='1/2 part pineapple을 추가합니다.')

measure_modify(281, ['2', 'slice'], 4, description='2개의 파인애플 슬라이스로 마무리하십시오. (granish)')

measure_append(286, ['fill to top'], 3, description='남은 용량은 coffee로 채웁니다.')

measure_append(287, ['fill'], 2, description='ginger ale로 glass를 채우십시오.')
measure_append(287, ['garnish'], 3, description='lime으로 장식하십시오.')
measure_append(287, ['cubes'], 4, description='Ice cube를 추가합니다.')

measure_modify(292, ['fill to top'], 1, description='남은 용량은 tea로 채우십시오.')

measure_modify(293, ['garnish'], 3, description='olive를 마지막에 추가하십시오.')

measure_modify(294, ['1/2'], 0, description='instrution에서도 구체적인 용량 정보가 없음으로, 1/2로 가정했습니다.')
measure_append(294, ['1/2'], 1, description='instruction에서도 구체적인 용량 정보가 없음으로, 1/2로 가정했습니다.')
measure_append(294, ['a little bit of'], 3, description='조금 추가해서 술에 맞을 더하는 용도로 사용합니다.')
measure_append(294, ['a little bit of'], 4, description='조금 추가해서 술에 맞을 더하는 용도로 사용합니다.')

measure_modify(295, ['1/5', 'part'], 5, description='1/5 part만큼 vodka를 추가합니다.')

measure_modify(297, ['5', 'whole'], 4, description='5개의 mini-marshmellow를 추가합니다.')


measure_modify(301, ['fill'], 1, description='coffee로 glass를 가득 채웁니다.')

measure_append(303, ['fill'], 5, description='carbonated water로 glass를 가득 채웁니다.')

measure_modify(304, ['1/2', 'piece'], 0, description='lime의 1/2를 glass에 추가합니다.')


measure_modify(309, ['1', 'garnish'], 5, description='cherry를 장식용으로 추가합니다.')

measure_modify(311, ['4-5', 'cubes'], 0, description='4~5개의 ice cube를 추가합니다.')

measure_append(314, ['fill'], 4, description='milk를 glass에 각득 채웁니다.')

measure_append(315, ['top up with'], 4, description='whipped cream을 추가하여 마무리합니다.')

measure_modify(316, ['1', 'whole'], 4, description='egg white 1개를 추가합니다.')
measure_append(316, ['fill'], 6, description='carbonated water로 glass를 채웁니다.')

measure_modify(321, ['1', 'garnish'], 4, description='cherry를 장식용으로 추가합니다.')
measure_modify(321, ['1', 'wedge'], 5, description='orange 조각을 추가합니다.')


measure_append(322, ['top up with'], 2, description='instruction에도 정량적인 값을 알 수 없습니다. 대략 위에 추가하는 걸로 유추해서 했습니다.')
measure_append(322, ['top up with'], 3, description='instruction에도 정량적인 값을 알 수 없습니다. 대략 위에 추가하는 걸로 유추해서 했습니다.')

measure_modify(327, ['juice of', '1/4'], 1, description='1/4 lemon juice를 더합니다.')

measure_append(328, ['fill'], 3, description='coffee를 glass에 가득 채웁니다.')

measure_modify(333, ['3', 'whole'], 0, description='kiwi 3개를 추가합니다.')
measure_modify(333, ['1/2', 'whole'], 1, description='papaya 1/2조각을 추가합니다. 추가하는 량이 전체 한 과일의 1/2임으로 piece 대신 whole을 사용했습니다.')

measure_append(335, ['a little bit of'], 5, description='cranberry juice 약간을 추가합니다.')

measure_modify(337, ['fill to top'], 1, description='turkish apple tea로 남은 용량을 채웁니다.')
measure_modify(337, ['by taste'], 2, description='설탕은 취향에 따라서 추가합니다.')


measure_modify(338, ['1', 'whole'], 4, description='egg white 1개를 추가합니다.')
measure_append(338, ['fill'], 6, description='carbonated water로 glass를 채웁니다.')


measure_modify(340, ['2', 'whole'], 0, description='mango 2개를 추가합니다.')

measure_modify(343, ['4-6', 'cubes'], 1, description='4~6개의 ice cube를 추가합니다.')
measure_append(343, ['to taste'], 3, description='설탕은 취향에 따라 추가합니다.')
measure_append(343, ['by taste'], 4, description='lime 또한 취향에 따라 추가하도록 합니다.')
measure_append(343, ['to taste'], 5, description='소금또한 취향에 따라 조절합니다.')

measure_modify(345, ['1', 'wedge'], 2, description='레몬 조각을 추가합니다.')
measure_modify(345, ['by taste'], 3, description='instruction에도 정확한 정량은 없음으로 취향에 따르도록 합니다.')
measure_append(345, ['fill to top'], 5, description='151 proof rum 술을 추가합니다.')

measure_modify(346, ['1', 'dash'], 4, description='ground cayenne pepper를 대략 1ml정도 추가합니다.')

measure_modify(347, ['fill'], 2, description='seltzer water로 glass를 채웁니다.')
measure_append(347, ['1', 'twist of'], 4, description='lime peel을 추가합니다.')

measure_append(352, ['1/3'], 1, description='carboated water를 추가합니다.')
measure_append(352, ['1/3'], 2, description='gin을 추가합니다.')
measure_append(352, ['1/3'], 3, description='dry vermouth를 추가합니다.')
measure_append(352, ['by taste'], 4, description='instruction 에서도 설탕량을 구체적으로 알 수 없음으로 취향에 따르도록 합니다.')
measure_append(352, ['3', 'wedge'], 5, description='instruction에서도 정량을 알 수 없습니다. orange spiral 2조각으로 가정합니다.')
measure_append(352, ['twist of'], 6, description='lemon peel을 추가합니다.')

measure_modify(355, ['juice of', '1/2'], 1, description='1/2 lemon 착즙을 추가합니다.')
measure_modify(355, ['4', 'cubes'], 4, description='4개의 ice cube를 추가합니다.')


measure_modify(358, ['1', 'garnish'], 4, description='cherry를 장식용으로 추가합니다.')

measure_modify(360, ['1', 'whole'], 0, description='1개의 망고를 추가합니다.')
measure_modify(360, ['2', 'whole'], 1, description='2개의 오랜지를 추가합니다.')

measure_modify(361, ['2-3', 'cubes'], 3, description='2~3개의 ice cube를 추가합니다.')
measure_modify(361, ['1', 'garnish'], 4, description='cherry를 장식용으로 추가합니다.')

measure_append(362, ['a little bit of'], 4, description='instruction에도 정량적인 소금의 양을 알 수 없습니다. 대략적으로 조금의 소금을 쓰는 것으로 가정합니다.')

measure_modify(363, ['1', 'garnish'], 4, description='cherry를 장식용으로 추가합니다.')

measure_modify(364, ['1', 'garnish'], 2, description='olive를 장식으로 추가합니다.')

measure_modify(365, ['1', 'garnish'], 4, description='cherry를 장식용으로 추가합니다.')

measure_modify(366, ['3-4', 'whole'], 3, description='3~4개의 cardamom을 추가합니다.')
measure_modify(366, ['3', 'whole'], 4, description='3개의 clove를 추가합니다.')
measure_append(366, ['to taste'], 9, description='취향에따라 우유를 추가합니다.')

measure_modify(367, ['1', 'tsp'], 0, description='1개의 teaspoon만큼의 꿀을 추가합니다.')
measure_append(367, ['1', 'tsp'], 3, description='cocoa power 1 teaspoon을 추가합니다.')
measure_append(367, ["fill"], 1, description='coffee espresso로 glass를 채웁니다.')

measure_modify(368, ['1/3'], 1, description='pina colada mix를 1/3 만큼 추가합니다.')
measure_modify(368, ['1/3'], 2, description='froze daiquiri mix를 1/3 만큼 추가합니다.')

measure_modify(373, ['3/4'], 0, description='glass의 3/4만큼 추가합니다.')

measure_append(374, ['fill'], 6, description='carbonated water로 glass를 채웁니다.')

measure_append(375, ['top up with'], 4, description='whipped cream으로 마무리합니다.')

measure_modify(376, ['2-4', 'leaves'], 3, description='2~4개의 mint leaves를 추가합니다.')
measure_append(376, ['fill'], 5, description='soda water')


measure_modify(384, ['12', 'whole'], 2, description='12개의 cloves를 추가합니다.')
measure_modify(384, ['2', 'whole'], 3, description='2개의 cinnamon을 추가합니다.')
measure_modify(384, ['twist of'], 4, description='lemon peel을 추가합니다.')

measure_modify(388, ['1/3'], 3, description='1/3 정도의 Claret red wine을 추가합니다.')
measure_append(388, ['1/2', 'slice'], 5, description='슬라이스의 1/2크기 레몬으로 장식합니다.')
measure_append(388, ['1', 'garnish'], 6, description='cherry를 장식용으로 추가합니다.')


measure_modify(391, ['1', 'cubes'], 2, description='cube 형태의 sugar를 추가합니다.')

measure_append(393, ['fill to top'], 4, description='ginger ale로 glass를 채웁니다.')


measure_modify(395, ['6', 'inch'], 2, description='3개의 2인치의 orange peel을 사용합니다.')

measure_modify(396, ['cubes'], 4, description='ice cube를 추가합니다.')

measure_append(397, ['top up with'], 4, description='Soda water를 참가함으로써 마무리합니다.')

measure_modify(398, ['1', 'whole'], 4, description='oreo cookie 1개를 추가한다.')

measure_modify(403, ['1/2', 'whole'], 1, description='1/2개의 pineapple를 추가합니다.')

measure_modify(405, ['1', 'whole'], 3, description='egg white 1개를 추가합니다.')

measure_append(408, ['cubes'], 4, description='ice cube를 추가합니다.')

measure_append(414, ['a little bit of'], 5, description='nutmeg 약간을 추가합니다.')

measure_append(416, ['cubes'], 4, description='ice cube를 추가합니다.')

measure_append(420, ['fill to top'], 3, description='lemon-lime soda로 glass를 채웁니다.')

measure_append(427, ['1/3'], 2, description='instruction에서도 정확한 양을 알 수 없습니다. 1/3으로 가정했습니다.')
measure_append(427, ['2/3'], 3, description='instruction에서도 정확한 값을 알 수 없습니다. 2/3으로 가정했습니다.')

measure_append(428, ['cubes'], 3, description='ice cube를 추가합니다.')

measure_append(430, ['by taste'], 6, description='instruction에서도 정확한 양을 알 수 없습니다. 취향에 따라서 넣는 걸로 가정했습니다.')

measure_append(432, ['1', 'can'], 4, description='1 캔의 콜라 분량을 추가합니다.')

measure_append(434, ['fill to top'], 5, description='carboated water로 남은 용량을 채웁니다.')

measure_modify(436, ['1', 'whole'], 2, description='1개의 레몬을 여러개로 슬라이스해서 추가합니다.')
measure_modify(436, ['1', 'garnish'], 4, description='cherry를 장식용으로 추가합니다')
measure_modify(436, ['1', 'whole'], 5, description='1개의 오랜지를 여러개로 슬라이스해서 추가합니다.')

measure_modify(437, ['1', 'whole'], 2, description='1개의 레몬을 여러개의 wedge로 나누어서 추가합니다.')

measure_append(438, ['a little bit of'], 4, description='nutmeg를 약간 추가합니다.')

measure_modify(440, ['1', 'bottle'], 1, description='ginger ale의 큰 1병을 추가합니다.')
measure_modify(440, ['cubes'], 4, description='ice cube를 추가합니다.')

measure_modify(443, ['3', 'slice'], 3, description="orange의 slice를 추가하여 장식합니다. 정확한 slice 양이 불분명함으로 3개로 가정했습니다.")
measure_modify(443, ['1', 'garnish'], 4, description="cherry를 장식용으로 추가합니다.")

measure_append(448, ['1/3'], 3, description='instruction에서도 정확한 양을 알 수가 없습니다. 1/3이면 대략 맞을 듯 합니다.')
measure_append(448, ['1/3'], 4, description='instruction에서도 정확한 양을 알 수가 없습니다. 1/3이면 대략 맞을 듯 합니다.')

measure_append(449, ['3', 'whole'], 5, description='instruction에서도 정확한 양을 알 수가 없습니다. 3 whole로 가정합니다.')
measure_append(449, ['2', 'whole'], 6, description='instruction에서도 정확한 양을 알 수가 없습니다. 2 whole이면, 각 스틱을 기준으로 2개입니다.')

measure_append(450, ['to taste'], 7, description='instruction에서는 정확한 양 대신 조건이 주어졌습니다. 이 조건은 취향에 따른다고 해석하도록 합니다.')

measure_modify(453, ['1', 'sprig'], 4, description='1개의 mint 잎을 추가합니다.')

measure_modify(454, ['1', 'garnish'], 4, description='cheery를 장식용으로 추가합니다.')

measure_modify(455, ['1', 'fifth'], 0, description='fifth는 750ml의 병을 의미합니다.')

measure_append(457, ['fill up to'], 2, description='orange juice를 glass에 채웁니다.')

measure_append(459, ['1/3'], 3, description='instruction에서도 정확한 양을 알 수가 없습니다. 다른 재료를 통해 유추해보면 1/3정도 될 걸로 보입니다.')
measure_append(459, ['1/3'], 4, description='instruction에서도 정확한 양을 알 수가 없습니다. 다른 재료를 통해 유추해보면 1/3정도 될 걸로 보입니다.')

measure_append(462, ['fill'], 4, description='우유를 glass 끝가지 채웁니다.')
measure_append(462, ['a little bit of'], 5, description='nutmeg를 약간 추가합니다.')

measure_append(463, ['a little bit of'], 5, description='nutmeg를 약간 추가합니다.')

measure_append(467, ['fill to top'], 5, description='instruction에서는 구체적인 양이 지정되어 있지 않지만, 나머지 음료의 양이 정해져 있음으로 fill to top 해석합니다.')
measure_append(467, ['garnish'], 6, description='cheery로 장식합니다.')

measure_append(474, ['fill to top'], 4, description='orange juice로 glass를 채웁니다.')
measure_append(474, ['top up with'], 5, description='grenadine을 마지막에 추가해서 마무리합니다.')
measure_append(474, ['garnish'], 6, description='cheery로 장식합니다.')

measure_modify(475, ['2', 'whole'], 3, description='달걀 노른자 2개를 추가합니다.')

measure_modify(476, ['3', 'stick'], 3, description='3개의 각각 3inch씩의 cinnamon을 추가합니다.')

measure_append(477, ['fill'], 1, description='coffee로 glass를 채웁니다.')
measure_append(477, ['1', 'pinch'], 2, description='cinnammon 을 약간 추가합니다.')

measure_modify(482, ['8-10', 'whole'], 2, description='8-10개의 익은 strawberry를 추가합니다.')

measure_append(483, ['1', 'tsp'], 6, description='salt가 뭍은 잔에 다른 재료를 추가하는 방식으로 소개되어 있습니다. 정확한 양을 알기 어려우니 1 teaspoon으로 가정했습니다.')

measure_modify(485, ['1', 'whole'], 3, description='1개의 레몬을 모두 사용함으로 1 whole로 표현했습니다.')

measure_modify(488, ['1', 'piece'], 1, description='1개의 바나나를 추가합니다.')

measure_append(489, ['1', 'whole'], 7, description='정확한 양이 명시되어 있지 않지만, 1개의 레몬 사용으로 가정합니다.')
measure_append(489, ['fill to top'], 8, description='fresca를 남은 그릇의 용량까지 끝가지 채움니다.')

measure_append(490, ['fill to top'], 2, description="남은 그릇의 용량을 milk로 채움니다.")

measure_append(492, ['top up with'], 4, description='whipped cream으로 마무리합니다.')

measure_modify(493, ['1', 'whole'], 3, description='1개의 계란 흰자를 추가합니다.')
measure_append(493, ['fill to top'], 5, description='ginger ale을 남은 그릇의 용량에 채웁니다.')

measure_modify(494, ['1', 'garnish'], 4, description='cheery를 장식용으로 추가합니다.')

measure_modify(495, ['1/2'], 0, description='정확한 용량 표기를 알수 없어 1/2로 가정했습니다.')
measure_append(495, ['1/2'], 2, description='정확한 용량을 알 수 없어 1/2로 가정했습니다.')
measure_append(495, ['top up with'], 3, description='grenadine을 마지막에 추가하여 마무리합니다.')

measure_modify(496, ['fill'], 0, description='tequila로 가득 채웁니다.')

measure_append(499, ['1-2', 'tsp'], 4, description='1~2개의 teaspoon만큼의 설탕을 추가합니다.')
measure_append(499, ['top up with'], 5, description='whipped cream을 위에 추가합니다.')
measure_append(499, ['5-6', 'cubes'], 6, description='5~6개의 ice cube를 추가합니다.')


measure_modify(500, ['1/2'], 0, description='정확한 용량을 알기 어려워, 1/2로 가정하였습니다.')
measure_modify(500, ['1/2'], 1, description='정확한 용량을 알기 어려워, 1/2로 가정하였습니다.')
measure_append(500, ['to taste'], 2, description='설탕은 개인 취향에 따라 넣도록 합니다.' )
measure_append(500, ['2-3', 'whole'], 4, description='cardamom을 2~3개 정도 커피에 넣습니다.')

measure_append(501, ['2-3', 'leaves'], 5, description='mint leaves를 여러개 넣습니다.')

measure_modify(505, ['1', 'garnish'], 4, description='cherry를 장식용으로 추가합니다.')
measure_modify(505, ['1', 'slice'], 5, description = '장식용으로 orange slice 1개를 추가합니다.')

measure_modify(507, ['2', 'scoop'], 2, description='2개의 spoon 분량만큼 추가합니다. 문제는 spoon이 어떤 스푼인지 정확하지 않다는 점입니다. 따라서 scoop단위로 바꿔 사용합니다.')

measure_modify(510, ['1', 'garnish'], 5, description='cherry를 장식용으로 추가합니다.')

measure_append(514, ['fill to top'], 3, description='carbonated water로 glass를 채움니다.')

measure_modify(516, ['1', 'whole'], 4, description='1개의 계란 흰자를 추가합니다.')

measure_append(520, ['fill'], 2, description='topnic water를 거의 끝까지 채움니다.')

measure_append(521, ['cubes'], 4, description='ice cube를 추가합니다.')
measure_append(521, ['a little bit of'], 5, description='nutmeg를 약간 추가합니다.')


measure_modify(522, ['1', 'whole'], 2, description='olive를 마지막으로 넣습니다. (장식적인 느낌)')

measure_append(523, ['fill'], 2, description='Schweppes Russchian을 거의 glass에 가득 채웁니다.')

measure_modify(525, ['1', 'garnish'], 3, description='cherry를 장식용으로 추가합니다.')

measure_append(528, ['fill to top'], 3, description='light cream을 컵용량에 맞게 더합니다.')

measure_modify(531, ['2', 'whole'], 1, description='2개의 레몬을 추가합니다.' )
measure_modify(531, ['3', 'whole'], 3, description='3개의 orange를 추가합니다.')

measure_append(533, ['cubes'], 3, description='ice cube를 추가합니다.')

measure_modify(542, ['2-3', 'slice'], 3, description='2~3개의 lime lemon 슬라이스를 더합니다. 정확한 용량이 나와있지 않아, 2~3 슬라이스로 가정했습니다.')

measure_modify(543, ['1', 'can'], 0, description='coca-cola 1캔을 모두 사용합니다.')
measure_append(543, ['1/6'], 1, description='beer 1/6을 추가합니다. 총 음료계열 재료의 수로 나누었습니다.')
measure_append(543, ['1/6'], 2, description='root bear  1/6을 추가합니다. 총 음료계열 재료의 수로 나누었습니다.')
measure_append(543, ['1/6'], 3, description='lemonade 1/6을 추가합니다. 총 음료계열 재료의 수로 나누었습니다.')
measure_append(543, ['1/6'], 5, description='7-up 1/6을 추가합니다. 총 음료계열 재료의 수로 나누었습니다.')
measure_append(543, ['1/6'], 6, description='orange juice 1/6을 추가합니다. 총 음료계열 재료의 수로 나누었습니다.')
measure_append(543, ['1', 'whole'], 7, description='lemon의 경우 1개를 모두 사용하였습니다. 장식용도 포함합니다.')




In [27]:
# 작업 검증
for i in range(len(all_drinks)):
    # print("ALL DrinK: ", all_drinks['Measures'][i])
    # print("Compared Alcohol: ", compared_alcohol['Measures'][i])
    # print("Ingredients: ", all_drinks['Ingredients'][i])
    if (len(all_drinks['Ingredients'][i]) != len(all_drinks['Measures'][i])):
        print("index: ", i)
        print("number of Ingredients: ", len(all_drinks['Ingredients'][i]))
        print("number of Measures: ", len(all_drinks['Measures'][i]))
        print('Check again!')

In [28]:
# indices to check
check_list = [8, 9, 10, 11, 19, 21, 22, 23, 25, 29, 31, 32, 33, 37, 38, 41, 42, 45, 52, 
              54, 55, 57, 59, 62, 69, 74, 75, 76, 81, 84, 85, 88, 89, 90, 98, 99, 100, 
              101, 103, 16, 110, 112, 114, 116, 124, 126, 127, 128, 129, 130, 131, 134, 
              135, 141, 149, 150, 152, 154, 155, 156, 159, 161, 162, 163, 167, 168, 170, 
              172, 175, 177, 178, 179, 180, 181, 186, 187, 189, 192, 193, 195, 197, 200, 
              202, 204, 210, 212, 213, 217, 222, 223, 224, 225, 226, 231, 232, 233, 234, 
              235, 246, 247, 251, 252, 254, 256, 258, 262, 263, 264, 265, 266, 268, 269, 
              270, 274, 280, 281, 286, 287, 292, 293, 294, 295, 297, 301, 303, 304, 309, 
              311, 314, 315, 316, 321, 322, 327, 328, 333, 335, 337, 338, 340, 343, 345, 
              346, 347, 352, 355, 358, 360, 361, 362, 363, 364, 365, 366, 367, 368, 373, 
              374, 375, 376, 384, 388, 391, 393, 395, 396, 398, 403, 405, 408, 414, 416, 
              420, 427, 428, 430, 432, 434, 436, 437, 438, 440, 443, 448, 449, 450, 453,
              454, 457, 459, 462, 463, 467, 474, 475, 476, 477, 482, 483, 485, 488, 489,
              490, 492, 493, 494, 495, 496, 499, 500, 501, 505, 507, 510, 514, 516, 520,
              521, 522, 523, 525, 528, 531, 533, 542, 543]

# all_drinks = pd.read_pickle('./preprocessed_alcohol_v3.pkl')


In [29]:
for index in check_list:
    drink = all_drinks['Measures'][index]
    drink_c = compared_alcohol['Measures'][index]
    print("index: ", index)
    print("drink: ", drink)
    print("original: ", drink_c)
    print()
    

index:  8
drink:  [[['1/2', 'oz'], 1], [['1/2', 'oz'], 2], [['1/2', 'oz'], 3], [['1/2', 'oz'], 4], [['1/2', 'oz'], 5], [['1-2', 'dash'], 7], [['1', 'wedge'], 8], [['1/2', 'oz'], 6], [['1', 'garnish'], 9]]
original:  [['1/2  oz ', 1], ['1/2  oz ', 2], ['1/2  oz ', 3], ['1/2  oz ', 4], ['1/2  oz ', 5], ['1-2  dash ', 7], ['1  wedge ', 8]]

index:  9
drink:  [[['2-3', 'oz'], 1], [['1', 'can'], 2]]
original:  [['2-3  oz', 1]]

index:  10
drink:  [[['5/2', 'oz'], 1], [['1', 'splash'], 2], [['fill'], 3]]
original:  [['2 1/2  oz ', 1], ['1  splash ', 2], [' fill with ', 3]]

index:  11
drink:  [[['70', 'ml'], 1], [['70', 'ml'], 2], [['70', 'ml'], 3], [['70', 'ml'], 4], [['70', 'ml'], 5]]
original:  []

index:  19
drink:  [[['1', 'shot'], 1], [['1', 'shot'], 2], [['cubes'], 3], [['fill'], 4]]
original:  [['1  shot ', 1], ['1  shot ', 2], ['  cubes', 3]]

index:  21
drink:  [[['1', 'jigger'], 1], [['juice of', '1/2'], 2], [['cubes'], 3], [['1', 'garnish'], 4]]
original:  [['1  jigger ', 1], [' 

In [30]:
all_drinks.to_pickle('./preprocessed_alcohol_v3.pkl')